In [16]:
from pyspark.sql import SparkSession

from pyspark.sql import functions as f
#from pyspark.sql.functions import  sum, avg, min, max,  mean
from pyspark.sql.types import *
from pyspark.sql.functions import countDistinct, avg, stddev
from pyspark.sql.functions import col, monotonically_increasing_id

from ut_spr import ut_spr
from ut_spark import ut_spark
from ut_clean import ut_clean

spark =SparkSession.\
        builder. \
        appName("Aggregation"). \
        master("local").\
        getOrCreate()

In [17]:
import pprint as ppr
from pyspark.sql.functions import count, col



from pyspark.sql.types import \
        StructType,\
        StringType, \
        IntegerType, \
        DateType, \
        TimestampType,\
        FloatType

class ut_spark:
    """ a class of utitlies related to Spark"""

    @classmethod
    def list_to_schema( cls,  fieldlist):
     def get_datatype(field_type):
        f_type=field_type [0:3]
        if f_type =='str':
            return  StringType()
        elif f_type=='int':
            return IntegerType()
        elif f_type == "dat":
            return DateType()
        elif f_type=="boo":
            return  BooleanType()
        elif f_type=="flo":
            return  FloatType()
        else:
            return StringType()
         
      
     structSchema = StructType()
        
     for fieldTuple in fieldlist:
        structSchema.add(fieldTuple[0], get_datatype(fieldTuple[1]), True)

     return(structSchema)

    @staticmethod
    def schema_tupple_tolow(field_tuple_list):
        for field in field_tuple_list:
            print("('" + field[0].lower() + "','" + field[1] + "'),")

    @staticmethod
    def load_df_csv(spark, data_file):
      return spark.read\
       .option("header","true")\
       .csv(data_file)

    @staticmethod
    def load_df_csv_infr_schema(spark, data_file):
      return spark.read\
       .option("header","true")\
       .option("inferSchema","true")\
       .csv(data_file)

    @staticmethod
    def load_df_csv_schema(spark, data_file, schema):
      return spark.read\
       .option("header","true")\
       .schema(schema)\
       .csv(data_file)




#### Read flights data  to dataframe

In [18]:
# create schema 
ft_fields = [
('Year','int'),
('Month','int'),
('DayofMonth','int'),
('DayOfWeek','int'),
('DepTime','int'),
('CRSDepTime','int'),
('ArrTime','int'),
('CRSArrTime','int'),
('UniqueCarrier','str'),
('FlightNum','str'),
('TailNum','str'),
('ActualElapsedTime','int'),
('CRSElapsedTime','int'),
('AirTime','int'),
('ArrDelay','int'),
('DepDelay','int'),
('Origin','str'),
('Dest','str'),
('Distance','int'),
('TaxiIn','int'),
('TaxiOut','int'),
('Cancelled','int'),
('CancellationCode','str'),
('Diverted','str'),
('CarrierDelay','int'),
('WeatherDelay','int'),
('NASDelay','int'),
('SecurityDelay','int'),
('LateAircraftDelay','int')
]

ft_schema=ut_spark.list_to_schema(ft_fields )



#read flight data to dataframe
f_folder = 'C:\\demo\\capstone\\raw_data\\'
f_name ='2008.csv.bz2'
f_path = f_folder+f_name

df0=spark.read\
       .option("header","true")\
       .schema(ft_schema)\
       .csv(f_path)

In [19]:
#add 'ROW_ID' column

df1=df0.withColumn('ROW_ID', monotonically_increasing_id())

In [20]:
df1.count()

2389217

#### For each column, find the number of records that have 'null' values

In [66]:
with ut_spr.timer():
    ut_clean.chk_dataframe_null(df0)

Year:0
Month:0
DayofMonth:0
DayOfWeek:0
DepTime:64442
CRSDepTime:0
ArrTime:70096
CRSArrTime:0
UniqueCarrier:0
FlightNum:0
TailNum:42452
ActualElapsedTime:70096
CRSElapsedTime:407
AirTime:70096
ArrDelay:70096
DepDelay:64442
Origin:0
Dest:0
Distance:0
TaxiIn:70096
TaxiOut:64442
Cancelled:0
CancellationCode:2324775
Diverted:0
CarrierDelay:1804634
WeatherDelay:1804634
NASDelay:1804634
SecurityDelay:1804634
LateAircraftDelay:1804634
Elapsed:120.030267


#### Find number of distinct values for column 'origin', 'dest', 'CancellationCode'

In [15]:
collist=[ 'origin', 'dest', 'CancellationCode' ]

for cl in collist:
    ut_clean.grpby(df1, cl)



+------+-----+
|origin|count|
+------+-----+
|   HOU|18721|
|   HRL| 1484|
|   IAD|26487|
|   IND|14402|
|   ISP| 3586|
|   JAN| 4307|
|   JAX|11637|
|   LAS|59653|
|   LAX|73480|
|   LBB| 2905|
|   LIT| 5179|
|   MAF| 2388|
|   MCI|21534|
|   MCO|45460|
|   MDW|30475|
|   MHT| 6361|
|   MSY|13782|
|   OAK|22620|
|   OKC| 8966|
|   OMA| 8908|
+------+-----+
only showing top 20 rows

+----+-----+
|dest|count|
+----+-----+
| LIT| 5184|
| MAF| 2390|
| MCO|45447|
| MDW|30466|
| MSY|13785|
| OAK|22623|
| OKC| 8973|
| PHL|33037|
| PHX|69397|
| SAN|31594|
| SAT|15336|
| STL|20817|
| TPA|28031|
| TUL| 8048|
| AUS|17135|
| HOU|18722|
| LAS|59657|
| BWI|34572|
| JAX|11644|
| MCI|21532|
+----+-----+
only showing top 20 rows

+----------------+-------+
|CancellationCode|  count|
+----------------+-------+
|               A|  26075|
|               C|  12617|
|               B|  25744|
|               D|      6|
|            null|2324775|
+----------------+-------+

